In [1]:
import tensorflow as tf
import math
print(tf.__version__)
tf.set_random_seed(0)

1.10.0


In [2]:
import mnistdata
mnist = mnistdata.read_data_sets("data/fashion", one_hot=True, reshape=False)

Successfully unzipped train-images-idx3-ubyte.gz
Successfully unzipped train-labels-idx1-ubyte.gz
Successfully unzipped t10k-images-idx3-ubyte.gz
Successfully unzipped t10k-labels-idx1-ubyte.gz


In [3]:
mnist.train.labels.shape # one hot

(60000, 10)

In [4]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y_ = tf.placeholder(tf.float32, [None, 10])
lr = tf.placeholder(tf.float32) # variable learning rate
pkeep = tf.placeholder(tf.float32) # probability of keeping a node during dropout
step = tf.placeholder(tf.int32) # step for variable learning rate

d_conv1 = 6  # first convolutional layer output depth
d_conv2 = 12  # second convolutional layer output depth
d_conv3 = 24  # third convolutional layer
d_fc1 = 200  # fully connected layer 
d_fc2 = 10 

W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, d_conv1], stddev=0.1))
B_conv1 = tf.Variable(tf.constant(0.1, tf.float32, [d_conv1]))
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, d_conv1, d_conv2], stddev=0.1))
B_conv2 = tf.Variable(tf.constant(0.1, tf.float32, [d_conv2]))
W_conv3 = tf.Variable(tf.truncated_normal([4, 4, d_conv2, d_conv3], stddev=0.1))
B_conv3 = tf.Variable(tf.constant(0.1, tf.float32, [d_conv3]))

W_fc1 = tf.Variable(tf.truncated_normal([7*7*d_conv3, d_fc1], stddev=0.1))
B_fc1 = tf.Variable(tf.constant(0.1, tf.float32, [d_fc1]))
W_fc2 = tf.Variable(tf.truncated_normal([d_fc1, d_fc2], stddev=0.1))
B_fc2 = tf.Variable(tf.constant(0.1, tf.float32, [d_fc2]))

In [5]:
# The model
stride = 1  # output is 28x28
Y_conv1 = tf.nn.relu(tf.nn.conv2d(X, W_conv1, strides=[1, stride, stride, 1], padding='SAME') + B_conv1)
stride = 2  # output is 14x14
Y_conv2 = tf.nn.relu(tf.nn.conv2d(Y_conv1, W_conv2, strides=[1, stride, stride, 1], padding='SAME') + B_conv2)
stride = 2  # output is 7x7
Y_conv3 = tf.nn.relu(tf.nn.conv2d(Y_conv2, W_conv3, strides=[1, stride, stride, 1], padding='SAME') + B_conv3)

# reshape the output from the third convolution for the fully connected layer
YY = tf.reshape(Y_conv3, shape=[-1, 7*7*d_conv3])

Y_fc1 = tf.nn.relu(tf.matmul(YY, W_fc1) + B_fc1)
YY_fc1 = tf.nn.dropout(Y_fc1, pkeep)
Y_fc2 = tf.matmul(YY_fc1, W_fc2) + B_fc2
Y = tf.nn.softmax(Y_fc2)

In [6]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Y_fc2, labels=Y_)
cross_entropy = tf.reduce_mean(cross_entropy)*100

correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# exponential_decay(learning_rate, global_step, decay_steps, decay_rate)
# decayed_learning_rate = learning_rate * decay_rate^{global_step/decay_steps}
lr = 1e-4 +  tf.train.exponential_decay(0.003, step, 2000, 1/math.e)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [7]:
test_X, test_Y = mnist.test.next_batch(10000)
for i in range(10000):
    batch_X, batch_Y = mnist.train.next_batch(100)
    if i % 1000 == 0:
        train_accuracy = accuracy.eval(session=sess, feed_dict={X:batch_X, Y_:batch_Y, pkeep:1.0})
        test_accuracy = accuracy.eval(session=sess, feed_dict={X:test_X, Y_:test_Y, pkeep:1.0})
        print("step %5d, train_accuracy = %.4g %% test_accuracy = %.4g %%" % (i, train_accuracy*100, test_accuracy*100))
    train_step.run(session=sess, feed_dict={X:batch_X, Y_:batch_Y, pkeep:0.75, step:i})

step     0, train_accuracy = 13 % test_accuracy = 13.47 %
step  1000, train_accuracy = 83 % test_accuracy = 88.39 %
step  2000, train_accuracy = 91 % test_accuracy = 90.48 %
step  3000, train_accuracy = 98 % test_accuracy = 90.71 %
step  4000, train_accuracy = 90 % test_accuracy = 91.17 %
step  5000, train_accuracy = 95 % test_accuracy = 91.56 %
step  6000, train_accuracy = 99 % test_accuracy = 91.48 %
step  7000, train_accuracy = 90 % test_accuracy = 91.43 %
step  8000, train_accuracy = 96 % test_accuracy = 91.35 %
step  9000, train_accuracy = 99 % test_accuracy = 91.64 %


In [7]:
# after restart kernel
test_X, test_Y = mnist.test.next_batch(10000)
for i in range(10000):
    batch_X, batch_Y = mnist.train.next_batch(100)
    if i % 1000 == 0:
        train_accuracy = accuracy.eval(session=sess, feed_dict={X:batch_X, Y_:batch_Y, pkeep:1.0})
        test_accuracy = accuracy.eval(session=sess, feed_dict={X:test_X, Y_:test_Y, pkeep:1.0})
        print("step %5d, train_accuracy = %.4g %% test_accuracy = %.4g %%" % (i, train_accuracy*100, test_accuracy*100))
    train_step.run(session=sess, feed_dict={X:batch_X, Y_:batch_Y, pkeep:0.5, step:i})

step     0, train_accuracy = 13 % test_accuracy = 13.47 %
step  1000, train_accuracy = 81 % test_accuracy = 87.78 %
step  2000, train_accuracy = 95 % test_accuracy = 90.1 %
step  3000, train_accuracy = 96 % test_accuracy = 90.71 %
step  4000, train_accuracy = 86 % test_accuracy = 90.88 %
step  5000, train_accuracy = 94 % test_accuracy = 91.24 %
step  6000, train_accuracy = 96 % test_accuracy = 91.35 %
step  7000, train_accuracy = 88 % test_accuracy = 91.41 %
step  8000, train_accuracy = 94 % test_accuracy = 91.41 %
step  9000, train_accuracy = 97 % test_accuracy = 91.52 %
